#Telegram Bot Uygulaması
**Döviz takibi yapabilmek tasarlanmıştır.**

####Kütüphanelerin ve Chromium indirme


#####Chromium İndirilmesi

In [ ]:
!apt-get update
!apt install chromium-chromedriver

#####Selenium Kütüphanesi İndirilmesi

In [ ]:
!pip install selenium

#####Telegram Bot Kütüphanesi İndirilmesi

In [ ]:
!pip install pyTelegramBotAPI

####Kullanılacak Kütüphaneneler

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import telebot
import time

#####Selenium ve Telebot Tanımlaması

In [ ]:
client_secenekleri = webdriver.ChromeOptions()
client_secenekleri.add_argument('--headless')
client_secenekleri.add_argument('--no-sandbox')
client_secenekleri.add_argument('--disable-dev-shm-usage')
Chromium = webdriver.Chrome('chromedriver',chrome_options=client_secenekleri)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
BotToken = "BOT_TOKEN"     
ChatID = "CHAT_ID"      
dovizBot = telebot.TeleBot(BotToken)

#####Kur Bilgisi Fonsiyonu

In [ ]:
def KurBilgisi():

  kur = ""
  Chromium.get("https://www.doviz.com/kripto-paralar/bitcoin")
  BTC = Chromium.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div/div[3]/div[1]/div[2]')
  btc = BTC.text
  Chromium.get("https://kur.doviz.com/serbest-piyasa/amerikan-dolari")
  USD = Chromium.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div[2]/div[1]/div[2]')
  usd = USD.text + " TL"
  Chromium.get("https://kur.doviz.com/serbest-piyasa/euro")
  EURO = Chromium.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div[2]/div[1]/div[2]')
  euro = EURO.text + " TL"
  Chromium.get("https://altin.doviz.com/gram-altin")
  GRAM_Altin = Chromium.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div[2]/div[1]/div[2]')
  gram = GRAM_Altin.text + " TL"

  kur_bilgileri = {
    "BTC":btc,
    "USD":usd,
    "EURO":euro,
    "GRAM ALTIN":gram  
  }

  for x,y in kur_bilgileri.items():
    kur += x + " = " + y + "\n----------------------------------\n"

  return kur

In [ ]:
def KurDegisim():
  while 1:
    global usdDegisimEski
    Chromium.get("https://kur.doviz.com/serbest-piyasa/amerikan-dolari")
    usdDegisim = Chromium.find_element(By.XPATH,'/html/body/div[3]/div[2]/div[1]/div[1]/div[2]/div[1]/div[3]/span[1]')
    usdDegisimYeni = float(usdDegisim.text[1:].replace(",","."))
    
    if usdDegisimEski > usdDegisimYeni:
      dovizBot.send_message(ChatID,"Dolar %"+ str(usdDegisimYeni) +" değerinde anlık DÜŞÜŞ yaşadı")
    if usdDegisimEski < usdDegisimYeni:
      dovizBot.send_message(ChatID,"Dolar %"+ str(usdDegisimYeni) +" değerinde YÜKSELİŞ oldu.")

    usdDegisimEski = usdDegisimYeni
    time.sleep(18)

#####Bot İstek Fonksiyonları


In [ ]:
@dovizBot.message_handler(commands=['kur'])
def kur(message):
 dovizBot.reply_to(message, KurBilgisi())

@dovizBot.message_handler(commands=['start','h'])
def start(message):
 if message.text == "/start":
    dovizBot.send_message(message.chat.id,"Döviz Takip Botuna Hoşgeldiniz!!\nYardım ve Komutlar için;\n /h") 
 if message.text == "/h":
    dovizBot.reply_to(message,"Bot komutları : /start, /h, /kur\n\n/kur : Anlık Döviz Bilgisi")

##### . . : : ANA KISIM : : . . 

In [ ]:
dovizBot.polling(skip_pending=True)